# 2nd Order Markov Chain Training on 10 Users' Trajectories

This notebook:
- Loads 10 users' trajectories (same as HMM/GNN/Fusion models)
- Removes consecutive duplicates (AAABCDCCABB → ABCDCAB) for each user
- Creates sequences of length 50 from all users
- Trains a 2nd order Markov chain model (P(X_t | X_{t-1}, X_{t-2}))
- Evaluates all 4 metrics: Accuracy, Precision & Recall, Top-K Accuracy, MPD


## Section 1 — Imports & Setup


In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from tqdm import tqdm
from haversine import haversine
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)

# Paths
BASE_PATH = "/home/root495/Inexture/Location Prediction Update"
PROCESSED_PATH = BASE_PATH + "/data/processed/"
SEQUENCES_FILE = PROCESSED_PATH + "place_sequences.json"
GRID_METADATA_FILE = PROCESSED_PATH + "grid_metadata.json"
CLEANED_WITH_PLACES_FILE = PROCESSED_PATH + "cleaned_with_places.csv"
OUTPUT_PATH = BASE_PATH + "/notebooks/"
MODELS_PATH = BASE_PATH + "/models/"
RESULTS_PATH = BASE_PATH + "/results/"
MODEL_SAVE_PATH = MODELS_PATH + "markov_chain_2order_model.pkl"
RESULTS_SAVE_PATH = RESULTS_PATH + "markov_chain_2order_results.json"

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

print("Libraries imported successfully!")


Libraries imported successfully!


## Section 2 — Load 10 Users' Trajectories


In [2]:
# Load place sequences
print("Loading place sequences...")
with open(SEQUENCES_FILE, 'r') as f:
    sequences_dict = json.load(f)

print(f"Total users available: {len(sequences_dict)}")

# Select first 10 users (same as HMM/GNN/Fusion models)
user_ids = list(sequences_dict.keys())
NUM_USERS = 10
selected_users = user_ids[:NUM_USERS]

print(f"\nSelected {NUM_USERS} users: {selected_users}")

# Load sequences for all selected users
user_sequences = {}
total_places = 0
for user_id in selected_users:
    seq = sequences_dict[user_id]
    user_sequences[user_id] = seq
    total_places += len(seq)
    print(f"  User {user_id}: {len(seq)} places")

print(f"\nTotal places across all {NUM_USERS} users: {total_places}")


Loading place sequences...
Total users available: 54

Selected 10 users: ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']
  User 000: 173817 places
  User 001: 108561 places
  User 005: 108967 places
  User 006: 31809 places
  User 009: 84573 places
  User 011: 90770 places
  User 014: 388051 places
  User 016: 89208 places
  User 019: 47792 places
  User 025: 628816 places

Total places across all 10 users: 1752364


## Section 3 — Preprocess: Remove Consecutive Duplicates

Remove consecutive duplicate locations for each user. Example: AAABCDCCABB → ABCDCAB

Only consecutive duplicates are removed. If a location appears again later (non-consecutive), it is kept.


In [3]:
def remove_consecutive_duplicates(sequence):
    """
    Remove consecutive duplicates from sequence.
    Example: [A, A, A, B, C, D, C, C, A, B, B] → [A, B, C, D, C, A, B]
    """
    if len(sequence) == 0:
        return sequence
    
    processed = [sequence[0]]  # Always keep first element
    
    for i in range(1, len(sequence)):
        # Only add if different from previous (not consecutive duplicate)
        if sequence[i] != sequence[i-1]:
            processed.append(sequence[i])
    
    return processed

# Apply consecutive duplicate removal to each user
processed_sequences = {}
total_original = 0
total_processed = 0

print("Processing users...")
for user_id in tqdm(selected_users, desc="Removing duplicates"):
    original_seq = user_sequences[user_id]
    processed_seq = remove_consecutive_duplicates(original_seq)
    processed_sequences[user_id] = processed_seq
    
    original_len = len(original_seq)
    processed_len = len(processed_seq)
    total_original += original_len
    total_processed += processed_len
    
    reduction = original_len - processed_len
    reduction_pct = (reduction / original_len * 100) if original_len > 0 else 0
    print(f"  User {user_id}: {original_len} → {processed_len} places ({reduction_pct:.1f}% reduction)")

print(f"\nSummary:")
print(f"  Total original places: {total_original}")
print(f"  Total after processing: {total_processed}")
print(f"  Total duplicates removed: {total_original - total_processed} ({((total_original - total_processed)/total_original*100):.1f}%)")


Processing users...


Removing duplicates:   0%|          | 0/10 [00:00<?, ?it/s]

  User 000: 173817 → 795 places (99.5% reduction)


Removing duplicates:  60%|██████    | 6/10 [00:00<00:00, 55.09it/s]

  User 001: 108561 → 186 places (99.8% reduction)
  User 005: 108967 → 283 places (99.7% reduction)
  User 006: 31809 → 103 places (99.7% reduction)
  User 009: 84573 → 17 places (100.0% reduction)
  User 011: 90770 → 125 places (99.9% reduction)
  User 014: 388051 → 766 places (99.8% reduction)
  User 016: 89208 → 124 places (99.9% reduction)
  User 019: 47792 → 120 places (99.7% reduction)


Removing duplicates: 100%|██████████| 10/10 [00:00<00:00, 28.85it/s]

  User 025: 628816 → 1568 places (99.8% reduction)

Summary:
  Total original places: 1752364
  Total after processing: 4087
  Total duplicates removed: 1748277 (99.8%)


## Section 4 — Create Sequences of Length 50

Split each user's processed sequence into fixed-length chunks of 50 events each.
Combine sequences from all users for training.


In [4]:
# Create sequences of fixed length 50
SEQUENCE_LENGTH = 50

# Use sliding windows for more training data (overlap helps with learning)
# Create overlapping sequences with step size of 25 (50% overlap)
all_sequences = []
step_size = 25  # Overlap of 50%

print("Creating sequences from all users...")
for user_id in tqdm(selected_users, desc="Processing users"):
    processed_seq = processed_sequences[user_id]
    user_sequences_list = []
    
    for i in range(0, len(processed_seq) - SEQUENCE_LENGTH + 1, step_size):
        chunk = processed_seq[i:i+SEQUENCE_LENGTH]
        if len(chunk) == SEQUENCE_LENGTH:  # Only full-length sequences
            user_sequences_list.append(chunk)
    
    all_sequences.extend(user_sequences_list)
    print(f"  User {user_id}: {len(user_sequences_list)} sequences")

print(f"\nTotal sequences created: {len(all_sequences)}")
print(f"Total events in sequences: {sum(len(s) for s in all_sequences)}")

# Split into train/test (80/20)
split_idx = int(len(all_sequences) * 0.8)
train_sequences = all_sequences[:split_idx]
test_sequences = all_sequences[split_idx:]

print(f"\nTraining sequences: {len(train_sequences)}")
print(f"Test sequences: {len(test_sequences)}")

if len(test_sequences) == 0:
    # If no test sequences, use last training sequence for testing
    test_sequences = [train_sequences[-1]]
    train_sequences = train_sequences[:-1]
    print(f"Adjusted: Training={len(train_sequences)}, Test=1 (using last training sequence)")


Creating sequences from all users...


Processing users: 100%|██████████| 10/10 [00:00<00:00, 13684.52it/s]

  User 000: 30 sequences
  User 001: 6 sequences
  User 005: 10 sequences
  User 006: 3 sequences
  User 009: 0 sequences
  User 011: 4 sequences
  User 014: 29 sequences
  User 016: 3 sequences
  User 019: 3 sequences
  User 025: 61 sequences

Total sequences created: 149
Total events in sequences: 7450

Training sequences: 119
Test sequences: 30


## Section 5 — Encode Sequences

Encode place_ids to integers for Markov chain training.


In [5]:
# Encode sequences to integers
print("Encoding sequences...")
le = LabelEncoder()

# Flatten all sequences for encoding
all_places = [place for seq in train_sequences + test_sequences for place in seq]
le.fit(all_places)

n_states = len(le.classes_)
print(f"Unique places across all users: {n_states}")

# Encode training sequences
train_encoded = []
for seq in train_sequences:
    encoded = le.transform(seq).tolist()
    train_encoded.append(encoded)

# Encode test sequences
test_encoded = []
for seq in test_sequences:
    encoded = le.transform(seq).tolist()
    test_encoded.append(encoded)

print(f"Encoded {len(train_encoded)} training sequences")
print(f"Encoded {len(test_encoded)} test sequences")

# Create mapping from encoded ID to original place_id for coordinate lookup
encoded_to_placeid = {}
for place_id in le.classes_:
    encoded_id = le.transform([place_id])[0]
    encoded_to_placeid[int(encoded_id)] = place_id

print(f"Created mapping for {len(encoded_to_placeid)} place IDs")


Encoding sequences...
Unique places across all users: 303
Encoded 119 training sequences
Encoded 30 test sequences
Created mapping for 303 place IDs


## Section 6 — Build 2nd Order Transition Matrix

Build transition probability matrix: P(X_t | X_{t-1}, X_{t-2})

Count all triplets (state_{t-2}, state_{t-1}, state_t) from training sequences and convert to probabilities.


In [6]:
# Build 2nd order transition matrix
# Structure: transitions[(prev2, prev1)][next_state] = count
print("Building 2nd order transition matrix from training sequences...")

transitions_2order = defaultdict(lambda: defaultdict(int))  # 2nd order transitions
transitions_1order = defaultdict(lambda: defaultdict(int))  # 1st order transitions (for fallback)
state_counts = Counter()  # Overall state frequencies (for fallback)

# Count all transitions from training sequences
total_transitions_2order = 0
total_transitions_1order = 0

for seq in tqdm(train_encoded, desc="Counting transitions"):
    # Count 2nd order transitions: (state_{t-2}, state_{t-1}) -> state_t
    for i in range(2, len(seq)):
        prev2 = seq[i-2]
        prev1 = seq[i-1]
        next_state = seq[i]
        transitions_2order[(prev2, prev1)][next_state] += 1
        total_transitions_2order += 1
    
    # Count 1st order transitions: state_{t-1} -> state_t (for fallback)
    for i in range(1, len(seq)):
        prev1 = seq[i-1]
        next_state = seq[i]
        transitions_1order[prev1][next_state] += 1
        total_transitions_1order += 1
    
    # Count state frequencies
    for state in seq:
        state_counts[state] += 1

print(f"\nTransition counts:")
print(f"  2nd order transitions: {total_transitions_2order}")
print(f"  1st order transitions: {total_transitions_1order}")
print(f"  Unique (prev2, prev1) pairs: {len(transitions_2order)}")
print(f"  Unique prev1 states: {len(transitions_1order)}")

# Convert counts to probabilities for 2nd order
print("\nConverting to probabilities...")
transition_probs_2order = {}
for (prev2, prev1), next_dict in transitions_2order.items():
    total = sum(next_dict.values())
    transition_probs_2order[(prev2, prev1)] = {
        next_state: count / total for next_state, count in next_dict.items()
    }

# Convert counts to probabilities for 1st order (fallback)
transition_probs_1order = {}
for prev1, next_dict in transitions_1order.items():
    total = sum(next_dict.values())
    transition_probs_1order[prev1] = {
        next_state: count / total for next_state, count in next_dict.items()
    }

# Get most frequent state (for final fallback)
most_frequent_state = state_counts.most_common(1)[0][0] if state_counts else None

print(f"Built transition probability matrices:")
print(f"  2nd order: {len(transition_probs_2order)} (prev2, prev1) pairs")
print(f"  1st order: {len(transition_probs_1order)} prev states (for fallback)")
print(f"  Most frequent state: {most_frequent_state} (count: {state_counts[most_frequent_state]})")

# Validate: Check that probabilities sum to 1.0 for a few examples
print("\nValidating probability distributions (checking first 3):")
for i, ((prev2, prev1), probs) in enumerate(list(transition_probs_2order.items())[:3]):
    total_prob = sum(probs.values())
    print(f"  ({prev2}, {prev1}): sum = {total_prob:.6f} ({len(probs)} next states)")

print("\n2nd order transition matrix built successfully!")


Building 2nd order transition matrix from training sequences...


Counting transitions: 100%|██████████| 119/119 [00:00<00:00, 6816.19it/s]


Transition counts:
  2nd order transitions: 5712
  1st order transitions: 5831
  Unique (prev2, prev1) pairs: 604
  Unique prev1 states: 286

Converting to probabilities...
Built transition probability matrices:
  2nd order: 604 (prev2, prev1) pairs
  1st order: 286 prev states (for fallback)
  Most frequent state: 220 (count: 1602)

Validating probability distributions (checking first 3):
  (219, 220): sum = 1.000000 (13 next states)
  (220, 219): sum = 1.000000 (8 next states)
  (219, 213): sum = 1.000000 (2 next states)

2nd order transition matrix built successfully!


In [7]:
def predict_next_location(history, use_patterns=True):
    """
    Predict next location using 2nd order Markov chain.
    
    Args:
        history: List of encoded states (integers)
        use_patterns: Whether to use transition probabilities
    
    Returns:
        Predicted next state (encoded integer) or None
    """
    if len(history) == 0:
        return most_frequent_state if most_frequent_state is not None else None
    
    if not use_patterns:
        return most_frequent_state if most_frequent_state is not None else None
    
    # Strategy 1: Use 2nd order transition if history length >= 2
    if len(history) >= 2:
        prev2 = history[-2]
        prev1 = history[-1]
        key = (prev2, prev1)
        
        if key in transition_probs_2order:
            next_probs = transition_probs_2order[key]
            if next_probs:
                # Return most likely next state
                most_likely = max(next_probs.items(), key=lambda x: x[1])[0]
                return int(most_likely)
    
    # Strategy 2: Fallback to 1st order if history length == 1
    if len(history) == 1:
        prev1 = history[-1]
        if prev1 in transition_probs_1order:
            next_probs = transition_probs_1order[prev1]
            if next_probs:
                most_likely = max(next_probs.items(), key=lambda x: x[1])[0]
                return int(most_likely)
    
    # Strategy 3: Final fallback to most frequent state
    return most_frequent_state if most_frequent_state is not None else None


def predict_top_k(history, k=5, use_patterns=True):
    """
    Get top-K most likely next locations using 2nd order Markov chain.
    
    Args:
        history: List of encoded states (integers)
        k: Number of top predictions to return
        use_patterns: Whether to use transition probabilities
    
    Returns:
        List of top-K encoded states
    """
    if len(history) == 0:
        # Return top-K most frequent states
        top_states = [state for state, _ in state_counts.most_common(k)]
        return top_states[:k] if top_states else []
    
    if not use_patterns:
        top_states = [state for state, _ in state_counts.most_common(k)]
        return top_states[:k] if top_states else []
    
    # Get probability distribution over all possible next states
    next_state_probs = {}
    
    # Strategy 1: Use 2nd order transition if history length >= 2
    if len(history) >= 2:
        prev2 = history[-2]
        prev1 = history[-1]
        key = (prev2, prev1)
        
        if key in transition_probs_2order:
            next_state_probs = transition_probs_2order[key].copy()
    
    # Strategy 2: If no 2nd order or incomplete, fallback to 1st order
    if not next_state_probs and len(history) >= 1:
        prev1 = history[-1]
        if prev1 in transition_probs_1order:
            next_state_probs = transition_probs_1order[prev1].copy()
    
    # Strategy 3: If still no probabilities, use state frequencies (normalized)
    if not next_state_probs:
        total_count = sum(state_counts.values())
        if total_count > 0:
            next_state_probs = {
                state: count / total_count 
                for state, count in state_counts.items()
            }
    
    # Return top-K states sorted by probability
    if next_state_probs:
        sorted_states = sorted(next_state_probs.items(), key=lambda x: x[1], reverse=True)
        top_k_states = [int(state) for state, prob in sorted_states[:k]]
        return top_k_states
    
    # Final fallback: most frequent states
    top_states = [state for state, _ in state_counts.most_common(k)]
    return top_states[:k] if top_states else []


print("Prediction functions defined successfully!")
print("\nFunction summary:")
print("  - predict_next_location(history, use_patterns=True): Returns single most likely next state")
print("  - predict_top_k(history, k=5, use_patterns=True): Returns top-K most likely next states")


Prediction functions defined successfully!

Function summary:
  - predict_next_location(history, use_patterns=True): Returns single most likely next state
  - predict_top_k(history, k=5, use_patterns=True): Returns top-K most likely next states


In [8]:
# Save model
print("Saving model...")
model_data = {
    'transition_probs_2order': dict(transition_probs_2order),
    'transition_probs_1order': dict(transition_probs_1order),
    'state_counts': dict(state_counts),
    'most_frequent_state': most_frequent_state,
    'label_encoder': le,
    'encoded_to_placeid': encoded_to_placeid,
    'n_states': n_states
}

with open(MODEL_SAVE_PATH, 'wb') as f:
    pickle.dump(model_data, f)

print(f"Model saved to {MODEL_SAVE_PATH}")
print(f"Model contains:")
print(f"  - 2nd order transition probabilities: {len(transition_probs_2order)} pairs")
print(f"  - 1st order transition probabilities: {len(transition_probs_1order)} states")
print(f"  - State frequencies: {len(state_counts)} states")
print(f"  - LabelEncoder and mappings")


Saving model...
Model saved to /home/root495/Inexture/Location Prediction Update/models/markov_chain_2order_model.pkl
Model contains:
  - 2nd order transition probabilities: 604 pairs
  - 1st order transition probabilities: 286 states
  - State frequencies: 286 states
  - LabelEncoder and mappings


## Section 9 — Evaluation Setup

Prepare test sequences and helper functions for evaluation metrics.


In [9]:
# Use first test sequence for evaluation (same as HMM notebook for fair comparison)
test_sequence = test_encoded[0]
print(f"Test sequence length: {len(test_sequence)} events")

# Create test cases: history -> next location
test_cases = []
for i in range(1, len(test_sequence)):
    history = test_sequence[:i]
    true_next = test_sequence[i]
    test_cases.append((history, true_next))

print(f"Created {len(test_cases)} test cases")

# Load grid metadata and coordinates for MPD calculation
with open(GRID_METADATA_FILE, 'r') as f:
    grid_metadata = json.load(f)

df_places = pd.read_csv(CLEANED_WITH_PLACES_FILE)
place_coords = df_places.groupby('place_id')[['lat', 'lon']].first().to_dict('index')

print(f"Loaded coordinates for {len(place_coords)} places")

# Helper function to get coordinates from place_id
def place_id_to_coords(place_id, place_coords, grid_metadata):
    """Get coordinates from place_id"""
    if place_id is None:
        return None, None
    
    # Try to find in place_coords first
    if place_id in place_coords:
        return place_coords[place_id]['lat'], place_coords[place_id]['lon']
    
    # Fallback: calculate from grid if place_id has format "row_col"
    try:
        if "_" in str(place_id):
            row, col = map(int, str(place_id).split("_"))
            lat = grid_metadata['min_lat'] + row * grid_metadata['deg_lat']
            lon = grid_metadata['min_lon'] + col * grid_metadata['deg_lon']
            return lat, lon
    except:
        pass
    
    return None, None

print("Evaluation setup complete!")


Test sequence length: 50 events
Created 49 test cases
Loaded coordinates for 2073 places
Evaluation setup complete!


## Section 10 — Metric 1: Accuracy

Calculate accuracy: fraction of predictions that exactly match the true next location.


In [10]:
# Calculate Accuracy
print("Calculating Accuracy...")
predictions = []
true_labels = []

for history, true_next in tqdm(test_cases, desc="Making predictions"):
    pred = predict_next_location(history, use_patterns=True)
    if pred is not None:
        predictions.append(pred)
        true_labels.append(true_next)

# Calculate accuracy
if len(predictions) == 0:
    print("ERROR: No predictions were made!")
    accuracy = 0
    correct = 0
    total = 0
else:
    correct = sum(1 for p, t in zip(predictions, true_labels) if p == t)
    total = len(predictions)
    accuracy = correct / total if total > 0 else 0
    
    # Debug: Show first few predictions vs true
    print(f"\nDebug - First 5 predictions:")
    for i in range(min(5, len(predictions))):
        pred_place = encoded_to_placeid.get(predictions[i], "Unknown")
        true_place = encoded_to_placeid.get(true_labels[i], "Unknown")
        match = "✓" if predictions[i] == true_labels[i] else "✗"
        print(f"  {match} Pred: {predictions[i]} ({pred_place[:20]}) | True: {true_labels[i]} ({true_place[:20]})")

print(f"\n{'='*60}")
print(f"METRIC 1: ACCURACY")
print(f"{'='*60}")
print(f"Correct predictions: {correct}")
print(f"Total predictions: {total}")
print(f"Accuracy: {accuracy:.12f}")
print(f"{'='*60}")


Calculating Accuracy...


Making predictions: 100%|██████████| 49/49 [00:00<00:00, 119907.17it/s]


Debug - First 5 predictions:
  ✗ Pred: 219 (296_2075) | True: 213 (295_2076)
  ✗ Pred: 220 (296_2076) | True: 214 (295_2077)
  ✓ Pred: 213 (295_2076) | True: 213 (295_2076)
  ✗ Pred: 214 (295_2077) | True: 220 (296_2076)
  ✓ Pred: 219 (296_2075) | True: 219 (296_2075)

METRIC 1: ACCURACY
Correct predictions: 34
Total predictions: 49
Accuracy: 0.693877551020


## Section 11 — Metric 2: Precision & Recall

**Definition**: 
- **Precision**: How many predicted locations were actually correct, weighted by class frequency
- **Recall**: Out of all true next locations, how many you successfully predicted, weighted by class frequency

Measuring how trustworthy the model is with visited and predicted locations using weighted averages.


In [11]:
# Calculate Precision & Recall (Weighted)
print("Calculating Precision & Recall (Weighted)...")

if len(predictions) > 0:
    precision_weighted = precision_score(true_labels, predictions, average='weighted', zero_division=0)
    recall_weighted = recall_score(true_labels, predictions, average='weighted', zero_division=0)
else:
    precision_weighted = recall_weighted = 0

print(f"\n{'='*60}")
print(f"METRIC 2: PRECISION & RECALL")
print(f"{'='*60}")
print(f"Precision: {precision_weighted:.12f}")
print(f"Recall: {recall_weighted:.12f}")
print(f"{'='*60}")


Calculating Precision & Recall (Weighted)...

METRIC 2: PRECISION & RECALL
Precision: 0.730539301968
Recall: 0.693877551020


## Section 12 — Metric 3: Top-K Accuracy

**Definition**: The true next location is considered correct if it appears in the top K predicted locations.

Top-K Accuracy: If the true next position is included in the top-K predictions (K=1, 3, 5).


In [12]:
# Calculate Top-K Accuracy
print("Calculating Top-K Accuracy...")

k_values = [1, 3, 5]
top_k_results = {}

for k in k_values:
    correct_k = 0
    total_k = 0
    
    for history, true_next in tqdm(test_cases, desc=f"Top-{k}"):
        top_k_preds = predict_top_k(history, k=k, use_patterns=True)
        if top_k_preds:
            total_k += 1
            if true_next in top_k_preds:
                correct_k += 1
    
    top_k_accuracy = correct_k / total_k if total_k > 0 else 0
    
    top_k_results[k] = {
        'correct': correct_k,
        'total': total_k,
        'accuracy': top_k_accuracy
    }

print(f"\n{'='*60}")
print(f"METRIC 3: TOP-K ACCURACY")
print(f"{'='*60}")
for k in k_values:
    result = top_k_results[k]
    print(f"Top-{k} Accuracy: {result['accuracy']:.12f}")
print(f"{'='*60}")


Calculating Top-K Accuracy...


Top-5: 100%|██████████| 49/49 [00:00<00:00, 96443.40it/s]


METRIC 3: TOP-K ACCURACY
Top-1 Accuracy: 0.693877551020
Top-3 Accuracy: 0.918367346939
Top-5 Accuracy: 0.918367346939


## Section 13 — Metric 4: Mean Prediction Distance (MPD)

**Definition**: Average Haversine distance (in meters) between actual next location and predicted next location.

MPD Distance: Mean Prediction Distance — Mean actual distance visited from predicted location of next visit.


In [13]:
# Calculate Mean Prediction Distance (MPD)
print("Calculating Mean Prediction Distance (MPD)...")

distances = []
failed_conversions = 0

for history, true_next in tqdm(test_cases, desc="Calculating distances"):
    pred = predict_next_location(history, use_patterns=True)
    
    if pred is not None:
        # Convert encoded IDs back to place_ids
        pred_place_id = encoded_to_placeid.get(pred)
        true_place_id = encoded_to_placeid.get(true_next)
        
        if pred_place_id and true_place_id:
            # Get coordinates
            pred_lat, pred_lon = place_id_to_coords(pred_place_id, place_coords, grid_metadata)
            true_lat, true_lon = place_id_to_coords(true_place_id, place_coords, grid_metadata)
            
            if pred_lat is not None and true_lat is not None:
                # Calculate haversine distance
                try:
                    distance_m = haversine((pred_lat, pred_lon), (true_lat, true_lon)) * 1000
                    # Filter out unrealistic distances (likely coordinate errors)
                    if distance_m < 1000000:  # Less than 1000 km
                        distances.append(distance_m)
                    else:
                        failed_conversions += 1
                except:
                    failed_conversions += 1
            else:
                failed_conversions += 1
        else:
            failed_conversions += 1
    else:
        failed_conversions += 1

if failed_conversions > 0:
    print(f"Warning: {failed_conversions} distance calculations failed or were filtered")

mpd = np.mean(distances) if len(distances) > 0 else 0
mpd_median = np.median(distances) if len(distances) > 0 else 0
mpd_std = np.std(distances) if len(distances) > 0 else 0

print(f"\n{'='*60}")
print(f"METRIC 4: MEAN PREDICTION DISTANCE (MPD)")
print(f"{'='*60}")
print(f"MPD Distance: {mpd:.12f} meters")
print(f"Valid distance calculations: {len(distances)}/{len(test_cases)}")
print(f"{'='*60}")


Calculating Mean Prediction Distance (MPD)...


Calculating distances: 100%|██████████| 49/49 [00:00<00:00, 54.75it/s]


METRIC 4: MEAN PREDICTION DISTANCE (MPD)
MPD Distance: 3691.026849589245 meters
Valid distance calculations: 49/49


In [14]:
# Compile all results
results = {
    'num_users': NUM_USERS,
    'selected_users': selected_users,
    'preprocessing': {
        'total_original_places': total_original,
        'total_after_duplicate_removal': total_processed,
        'total_duplicates_removed': total_original - total_processed,
        'sequence_length': SEQUENCE_LENGTH,
        'total_sequences': len(all_sequences),
        'training_sequences': len(train_sequences),
        'test_sequences': len(test_sequences)
    },
    'model': {
        'unique_states': n_states,
        'order': 2,
        'model_type': '2nd_order_markov_chain',
        'num_2order_transitions': len(transition_probs_2order),
        'num_1order_transitions': len(transition_probs_1order)
    },
    'accuracy': {
        'value': accuracy,
        'correct': correct,
        'total': total
    },
    'precision_recall': {
        'precision': float(precision_weighted),
        'recall': float(recall_weighted)
    },
    'top_k_accuracy': {
        f'top_{k}_accuracy': float(top_k_results[k]['accuracy']) for k in k_values
    },
    'mpd_distance': {
        'mpd_distance_meters': float(mpd),
        'valid_calculations': len(distances)
    }
}

# Display summary
print(f"\n{'='*60}")
print(f"EVALUATION RESULTS SUMMARY")
print(f"{'='*60}")
print(f"\nNumber of users: {NUM_USERS}")
print(f"Users: {selected_users}")
print(f"Total original places: {total_original}")
print(f"After duplicate removal: {total_processed}")
print(f"Training sequences: {len(train_sequences)}")
print(f"Test sequences: {len(test_sequences)}")

print(f"\n1. ACCURACY")
print(f"   Accuracy: {accuracy:.12f}")

print(f"\n2. PRECISION & RECALL")
print(f"   Precision: {precision_weighted:.12f}")
print(f"   Recall: {recall_weighted:.12f}")

print(f"\n3. TOP-K ACCURACY")
for k in k_values:
    acc = top_k_results[k]['accuracy']
    print(f"   Top-{k} Accuracy: {acc:.12f}")

print(f"\n4. MEAN PREDICTION DISTANCE (MPD)")
print(f"   MPD Distance: {mpd:.12f} meters")

print(f"\n{'='*60}")

# Save results
with open(RESULTS_SAVE_PATH, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to {RESULTS_SAVE_PATH}")

# Create results DataFrame
results_df = pd.DataFrame({
    'Metric': [
        'Accuracy',
        'Precision',
        'Recall',
        'Top-1 Accuracy',
        'Top-3 Accuracy',
        'Top-5 Accuracy',
        'MPD Distance'
    ],
    'Value': [
        f"{accuracy:.12f}",
        f"{precision_weighted:.12f}",
        f"{recall_weighted:.12f}",
        f"{top_k_results[1]['accuracy']:.12f}",
        f"{top_k_results[3]['accuracy']:.12f}",
        f"{top_k_results[5]['accuracy']:.12f}",
        f"{mpd:.12f}"
    ]
})

print("\nResults Table:")
print(results_df.to_string(index=False))



EVALUATION RESULTS SUMMARY

Number of users: 10
Users: ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']
Total original places: 1752364
After duplicate removal: 4087
Training sequences: 119
Test sequences: 30

1. ACCURACY
   Accuracy: 0.693877551020

2. PRECISION & RECALL
   Precision: 0.730539301968
   Recall: 0.693877551020

3. TOP-K ACCURACY
   Top-1 Accuracy: 0.693877551020
   Top-3 Accuracy: 0.918367346939
   Top-5 Accuracy: 0.918367346939

4. MEAN PREDICTION DISTANCE (MPD)
   MPD Distance: 3691.026849589245 meters


Results saved to /home/root495/Inexture/Location Prediction Update/results/markov_chain_2order_results.json

Results Table:
        Metric             Value
      Accuracy    0.693877551020
     Precision    0.730539301968
        Recall    0.693877551020
Top-1 Accuracy    0.693877551020
Top-3 Accuracy    0.918367346939
Top-5 Accuracy    0.918367346939
  MPD Distance 3691.026849589245


In [15]:
# Update models_comparison.csv
comparison_file = RESULTS_PATH + "models_comparison.csv"

# Read existing comparison file
try:
    comparison_df = pd.read_csv(comparison_file)
    
    # Check if Markov Chain row already exists
    if 'Markov Chain' in comparison_df['Model'].values:
        # Update existing row
        mask = comparison_df['Model'] == 'Markov Chain'
        comparison_df.loc[mask, 'Accuracy'] = f"{accuracy:.12f}"
        comparison_df.loc[mask, 'Precision'] = f"{precision_weighted:.12f}"
        comparison_df.loc[mask, 'Recall'] = f"{recall_weighted:.12f}"
        comparison_df.loc[mask, 'Top-1 Accuracy'] = f"{top_k_results[1]['accuracy']:.12f}"
        comparison_df.loc[mask, 'Top-3 Accuracy'] = f"{top_k_results[3]['accuracy']:.12f}"
        comparison_df.loc[mask, 'Top-5 Accuracy'] = f"{top_k_results[5]['accuracy']:.12f}"
        comparison_df.loc[mask, 'MPD Distance (meters)'] = f"{mpd:.12f}"
        print("Updated existing Markov Chain row in models_comparison.csv")
    else:
        # Add new row
        new_row = pd.DataFrame({
            'Model': ['Markov Chain'],
            'Accuracy': [f"{accuracy:.12f}"],
            'Precision': [f"{precision_weighted:.12f}"],
            'Recall': [f"{recall_weighted:.12f}"],
            'Top-1 Accuracy': [f"{top_k_results[1]['accuracy']:.12f}"],
            'Top-3 Accuracy': [f"{top_k_results[3]['accuracy']:.12f}"],
            'Top-5 Accuracy': [f"{top_k_results[5]['accuracy']:.12f}"],
            'MPD Distance (meters)': [f"{mpd:.12f}"]
        })
        comparison_df = pd.concat([comparison_df, new_row], ignore_index=True)
        print("Added new Markov Chain row to models_comparison.csv")
    
    # Save updated comparison file
    comparison_df.to_csv(comparison_file, index=False)
    print(f"Updated {comparison_file}")
    
    # Display updated comparison
    print("\nUpdated Models Comparison:")
    print(comparison_df.to_string(index=False))
    
except FileNotFoundError:
    # Create new comparison file if it doesn't exist
    comparison_df = pd.DataFrame({
        'Model': ['Markov Chain'],
        'Accuracy': [f"{accuracy:.12f}"],
        'Precision': [f"{precision_weighted:.12f}"],
        'Recall': [f"{recall_weighted:.12f}"],
        'Top-1 Accuracy': [f"{top_k_results[1]['accuracy']:.12f}"],
        'Top-3 Accuracy': [f"{top_k_results[3]['accuracy']:.12f}"],
        'Top-5 Accuracy': [f"{top_k_results[5]['accuracy']:.12f}"],
        'MPD Distance (meters)': [f"{mpd:.12f}"]
    })
    comparison_df.to_csv(comparison_file, index=False)
    print(f"Created new {comparison_file}")
except Exception as e:
    print(f"Warning: Could not update models_comparison.csv: {e}")
    print("Results have been saved to JSON file. Please update CSV manually if needed.")


Added new Markov Chain row to models_comparison.csv
Updated /home/root495/Inexture/Location Prediction Update/results/models_comparison.csv

Updated Models Comparison:
       Model       Accuracy      Precision         Recall Top-1 Accuracy Top-3 Accuracy Top-5 Accuracy MPD Distance (meters)
         HMM       0.653061       0.605081       0.653061       0.653061       0.897959       0.918367           4364.404451
         GNN       0.504762       0.438886       0.504762       0.504762       0.691837       0.787075           3216.861429
      Fusion       0.498639        0.44425       0.498639       0.498639       0.768027       0.819728           5196.347567
Markov Chain 0.693877551020 0.730539301968 0.693877551020 0.693877551020 0.918367346939 0.918367346939     3691.026849589245
